In [185]:
import pickle
import pandas as pd
import numpy as np
import requests

In [186]:
url = "https://www.bundeswahlleiter.de/dam/jcr/72f186bb-aa56-47d3-b24c-6a46f5de22d0/btw17_kerg.csv"

In [187]:
zs_df = pd.read_csv(url, encoding = "ISO-8859-1", delimiter = ';', skiprows = 5)

In [188]:
zs_df = zs_df.iloc[2:] # dropping non-meaningful rows
zs_df = zs_df.reset_index(drop=True) # reset index
relevant_columns = [0, 1, 2, 17, 21, 25, 29, 33, 37, 41, 45] 
zs_df = zs_df.iloc[:, relevant_columns] # picking relevant columns
column_titles = ['Wahlkreis-Nr.', 'Bundesland', 'Bundesland-Nr.', 'Gesamt', 'CDU', 'SPD', 'Die Linke', 'Bündnis 90/Die Grünen', 'CSU', 'FDP', 'AFD']
zs_df.columns = column_titles

In [189]:
# convert columns w/ numbers to numeric
numeric_columns = ['Wahlkreis-Nr.', 'Bundesland-Nr.', 'Gesamt', 'CDU', 'SPD', 'Die Linke', 'Bündnis 90/Die Grünen', 'CSU', 'FDP', 'AFD']
for i in numeric_columns:
    zs_df[i] = pd.to_numeric(zs_df[i], errors = 'coerce')

In [190]:
# filtering out NaN-rows and rows with state-wide total votes
mask1 = zs_df[zs_df['Bundesland-Nr.'] == 99].index
mask2 = zs_df[zs_df['Bundesland-Nr.'].isnull()].index
zs_df = zs_df.drop(mask1 | mask2)

In [191]:
# adding voting shares
zs_df['CDU/CSU'] = np.where(zs_df['CSU'].isnull(), zs_df['CDU'], zs_df['CSU'])
zs_df['CDU/CSU Anteil'] = zs_df['CDU/CSU']/zs_df['Gesamt']
zs_df['SPD Anteil'] = zs_df['SPD']/zs_df['Gesamt']
zs_df['Die Linke Anteil'] = zs_df['Die Linke']/zs_df['Gesamt']
zs_df['Bündnis 90/Die Grünen Anteil'] = zs_df['Bündnis 90/Die Grünen']/zs_df['Gesamt']
zs_df['FDP Anteil'] = zs_df['FDP']/zs_df['Gesamt']
zs_df['AFD Anteil'] = zs_df['AFD']/zs_df['Gesamt']

In [193]:
zs_df.head()

,Wahlkreis-Nr.,Bundesland,Bundesland-Nr.,Gesamt,CDU,SPD,Die Linke,Bündnis 90/Die Grünen,CSU,FDP,AFD,CDU/CSU,CDU/CSU Anteil,SPD Anteil,Die Linke Anteil,Bündnis 90/Die Grünen Anteil,FDP Anteil,AFD Anteil
0,1.0,Flensburg â Schleswig,1.0,170465.0,58320.0,40388.0,14002.0,22304.0,NaN,18955.0,11653.0,58320.0,0.342123,1.0,0.082140,0.130842,0.111196,0.068360
1,2.0,Nordfriesland â Dithmarschen Nord,1.0,138071.0,52928.0,31120.0,8589.0,15144.0,NaN,18050.0,9030.0,52928.0,0.383339,1.0,0.062207,0.109683,0.130730,0.065401
2,3.0,Steinburg â Dithmarschen SÃ¼d,1.0,130878.0,47366.0,29756.0,8732.0,12960.0,NaN,17298.0,11180.0,47366.0,0.361910,1.0,0.066719,0.099024,0.132169,0.085423
3,4.0,Rendsburg-EckernfÃ¶rde,1.0,156267.0,56585.0,35766.0,9962.0,19337.0,NaN,19071.0,11578.0,56585.0,0.362105,1.0,0.063750,0.123743,0.122041,0.074091
4,5.0,Kiel,1.0,152069.0,40736.0,36208.0,15546.0,26143.0,NaN,17804.0,10504.0,40736.0,0.267878,1.0,0.102230,0.171915,0.117078,0.069074


In [194]:
with open('zs_df.pickle', 'wb') as handle:
    pickle.dump(zs_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [197]:
# check for matchability with se_df

with open('zs_df.pickle', 'rb') as handle:
    zs_df = pickle.load(handle)
    
with open('se_df.pickle', 'rb') as handle:
    se_df = pickle.load(handle)

In [205]:
zs_df.tail()

,Wahlkreis-Nr.,Bundesland,Bundesland-Nr.,Gesamt,CDU,SPD,Die Linke,Bündnis 90/Die Grünen,CSU,FDP,AFD,CDU/CSU,CDU/CSU Anteil,SPD Anteil,Die Linke Anteil,Bündnis 90/Die Grünen Anteil,FDP Anteil,AFD Anteil
322,295.0,Zollernalb â Sigmaringen,8.0,137727.0,52284.0,19446.0,7061.0,15798.0,NaN,18394.0,18814.0,52284.0,0.379621,1.0,0.051268,0.114705,0.133554,0.136604
325,296.0,SaarbrÃ¼cken,10.0,145430.0,41394.0,38322.0,21669.0,11748.0,NaN,11753.0,14042.0,41394.0,0.284632,1.0,0.149000,0.080781,0.080816,0.096555
326,297.0,Saarlouis,10.0,157605.0,52981.0,42476.0,20320.0,8598.0,NaN,12059.0,15506.0,52981.0,0.336163,1.0,0.128930,0.054554,0.076514,0.098385
327,298.0,St. Wendel,10.0,138698.0,50660.0,38368.0,15686.0,6662.0,NaN,9569.0,12627.0,50660.0,0.365254,1.0,0.113095,0.048032,0.068992,0.091040
328,299.0,Homburg,10.0,143525.0,44538.0,39729.0,17773.0,8109.0,NaN,11096.0,16745.0,44538.0,0.310315,1.0,0.123832,0.056499,0.077311,0.116670


In [204]:
se_df.tail()

,Land,Wahlkreis-Nr.,Wahlkreis-Name,Gemeinden am 31.12.2015 (Anzahl),Fläche am 31.12.2015 (km²),Bevölkerung am 31.12.2015 - Insgesamt (in 1000),Bevölkerung am 31.12.2015 - Deutsche (in 1000),Bevölkerung am 31.12.2015 - Ausländer (%),Bevölkerungsdichte am 31.12.2015 (Einwohner je km²),Zu- (+) bzw. Abnahme (-) der Bevölkerung 2015 - Geburtensaldo (je 1000 Einwohner),...,CSU,FDP,AFD,CDU/CSU,CDU/CSU Anteil,SPD Anteil,Die Linke Anteil,Bündnis 90/Die Grünen Anteil,FDP Anteil,AFD Anteil
311,Saarland,297,Saarlouis,18,"891,4","265,5","240,8","9,3","297,8","-5,7",...,NaN,12059.0,15506.0,52981.0,0.336163,1.0,0.128930,0.054554,0.076514,0.098385
312,Saarland,298,St. Wendel,16,"801,7","216,7","205,5","5,1","270,3",-7,...,NaN,9569.0,12627.0,50660.0,0.365254,1.0,0.113095,0.048032,0.068992,0.091040
313,Saarland,299,Homburg,12,"550,3","243,7",223,"8,5","442,9","-6,3",...,NaN,11096.0,16745.0,44538.0,0.310315,1.0,0.123832,0.056499,0.077311,0.116670
314,Saarland,910,Land insgesamt,52,"2568,7","995,6","904,3","9,2","387,6","-5,9",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
315,Deutschland,999,Insgesamt,11092,"357385,7","82175,7","73523,7","10,5","229,9","-2,3",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [201]:
se_df = se_df.join(zs_df.set_index('Wahlkreis-Nr.'), on = 'Wahlkreis-Nr.')

In [203]:
se_df.tail()

,Land,Wahlkreis-Nr.,Wahlkreis-Name,Gemeinden am 31.12.2015 (Anzahl),Fläche am 31.12.2015 (km²),Bevölkerung am 31.12.2015 - Insgesamt (in 1000),Bevölkerung am 31.12.2015 - Deutsche (in 1000),Bevölkerung am 31.12.2015 - Ausländer (%),Bevölkerungsdichte am 31.12.2015 (Einwohner je km²),Zu- (+) bzw. Abnahme (-) der Bevölkerung 2015 - Geburtensaldo (je 1000 Einwohner),...,CSU,FDP,AFD,CDU/CSU,CDU/CSU Anteil,SPD Anteil,Die Linke Anteil,Bündnis 90/Die Grünen Anteil,FDP Anteil,AFD Anteil
311,Saarland,297,Saarlouis,18,"891,4","265,5","240,8","9,3","297,8","-5,7",...,NaN,12059.0,15506.0,52981.0,0.336163,1.0,0.128930,0.054554,0.076514,0.098385
312,Saarland,298,St. Wendel,16,"801,7","216,7","205,5","5,1","270,3",-7,...,NaN,9569.0,12627.0,50660.0,0.365254,1.0,0.113095,0.048032,0.068992,0.091040
313,Saarland,299,Homburg,12,"550,3","243,7",223,"8,5","442,9","-6,3",...,NaN,11096.0,16745.0,44538.0,0.310315,1.0,0.123832,0.056499,0.077311,0.116670
314,Saarland,910,Land insgesamt,52,"2568,7","995,6","904,3","9,2","387,6","-5,9",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
315,Deutschland,999,Insgesamt,11092,"357385,7","82175,7","73523,7","10,5","229,9","-2,3",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
